# GSEA

Using `GSEApy` I perform GSEA for cluster 1 and 2 from the systems aging study.

In [2]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gseapy as gp
from gseapy import Biomart

# Data

In [3]:
read_supp_table_3 = lambda sheet: pd.read_excel(
    "data/Supplementary Table 3.xlsx", 
    sheet_name=sheet,
    )

df_c1_up = read_supp_table_3("Cluster 1 - upregulated ")
df_c1_down = read_supp_table_3("Cluster 1 - downregulated")
df_c2_up = read_supp_table_3("Cluster 2 - upregulated")
df_c2_down = read_supp_table_3("Cluster 2 - downregulated")

df_c1_up["logFC"] = df_c1_up["max(|logFC|)"]
df_c1_down["logFC"] = - df_c1_down["max(|logFC|)"]
df_c1 = pd.concat([df_c1_up, df_c1_down]).rename(columns={"ENTREZ": "entrezgene_id"})

df_c2_up["logFC"] = df_c2_up["max(|logFC|)"]
df_c2_down["logFC"] = - df_c2_down["max(|logFC|)"]
df_c2 = pd.concat([df_c2_up, df_c2_down]).rename(columns={"ENTREZ": "entrezgene_id"})

del df_c1_up, df_c1_down, df_c2_up, df_c2_down
print(f"{df_c1.shape=} | {df_c2.shape=}")

df_c1.shape=(1243, 4) | df_c2.shape=(547, 4)


In [4]:
assert df_c1["entrezgene_id"].isna().sum() == 0
assert df_c2["entrezgene_id"].isna().sum() == 0

## Prepare data for GSEApy

In [5]:
entrez_ids = df_c1["entrezgene_id"].tolist() + df_c2["entrezgene_id"].tolist()
entrez_ids = list(set(entrez_ids))
print(f"{len(entrez_ids)=}")

len(entrez_ids)=1682


In [6]:
# Split in 2 queries not to exceed the limit
bm = Biomart()

queries_1 = {
    'entrezgene_id': entrez_ids[:800],
}
queries_2 = {
    'entrezgene_id': entrez_ids[800:],
}

results_1 = bm.query(dataset='hsapiens_gene_ensembl',
                   attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id'],
                   filters=queries_1)
results_2 = bm.query(dataset='hsapiens_gene_ensembl',
                   attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id'],
                   filters=queries_2)
        
gene_ids_map = pd.concat([results_1, results_2])
gene_ids_map = gene_ids_map.dropna(axis=0)
assert gene_ids_map["entrezgene_id"].isna().sum() == 0
gene_ids_map.head(2)

,ensembl_gene_id,external_gene_name,entrezgene_id
0,ENSG00000236362,GAGE12F,100008586
1,ENSG00000200394,SNORA38B,100124536


In [7]:
df_c1 = pd.merge(df_c1, gene_ids_map, on="entrezgene_id", how="left")
df_c2 = pd.merge(df_c2, gene_ids_map, on="entrezgene_id", how="left")

df_c1.to_csv("data/df_c1.tsv", sep='\t', index=False)
df_c2.to_csv("data/df_c2.tsv", sep='\t', index=False)

print(f"{df_c1.shape=} | {df_c2.shape=}")

df_c1.shape=(1393, 6) | df_c2.shape=(601, 6)


In [8]:
assert all(df_c1.groupby("external_gene_name")["logFC"].nunique() == 1), "All external gene names have same logFC"
assert all(df_c2.groupby("external_gene_name")["logFC"].nunique() == 1), "All external gene names have same logFC"

c1_rnk = pd.DataFrame(df_c1.groupby("external_gene_name")["logFC"].mean()).sort_values("logFC", ascending=False)
c2_rnk = pd.DataFrame(df_c2.groupby("external_gene_name")["logFC"].mean()).sort_values("logFC", ascending=False)

# Run GSEApy Prerank

- We use this approach because we operate on a ranked, metric list of genes, and not the classical GSEA scenario of many samples per conditions, 2 conditions.
- Gene set names come from [Enrichr libraries](https://maayanlab.cloud/Enrichr/#libraries).
- Takes about 6 mins to run on laptop for C1
- Tag% - % gene hits (in filtered gene set) before (ES>0) or after (ES<0) peak in running ES score. Gives an indication of percentage of genes contributing to the ES.
- Gene% - % genes in the ranked list before (ES>0) or after (ES<0) the peak in the running ES. Indicates where in the list the ES is attained.
- to include GO and smaller sets, we do another analysis where we don't limit `min_size` to 5 

In [8]:
GENE_SETS = [
    'KEGG_2021_Human', 
    'GO_Biological_Process_2021'
    'MSigDB_Hallmark_2020',
    'Reactome_2022',
    'ChEA_2022',
]
print(f"{len(GENE_SETS)=}")

len(GENE_SETS)=4


In [9]:
def run_prerank_analysis(
    data, 
    gene_sets, 
    outdir, 
    min_set_size = 5
    ):
    res = gp.prerank(
        rnk=data,
        gene_sets=gene_sets,
        threads=4,
        min_size=min_set_size,
        max_size=1000,
        permutation_num=1000,
        outdir=outdir,
        seed=42,
        verbose=False
        )
    return res


def process_results(analysis_results, outdir) -> pd.DataFrame:
    df_res = analysis_results.res2d.copy()
    df_res["Gene_set"] = df_res["Term"].str.split("__").str[0]
    df_res["Term_clean"] = df_res["Term"].str.split("__").str[1]

    for head_N in [10, 30]:
        df_res.groupby("Gene_set").head(head_N).to_csv(f"{outdir}/head-{head_N}_by_geneset.tsv", sep='\t')

    df_res.to_csv(f"{outdir}/processed.tsv", sep='\t')
    return df_res

C1

In [10]:
pre_res_c1 = run_prerank_analysis(c1_rnk, GENE_SETS, "results/prerank_c1")
df_res_c1 = process_results(pre_res_c1, "results/prerank_c1")

2022-10-31 19:42:21,588 No supported gene_sets: GO_Biological_Process_2021MSigDB_Hallmark_2020


C1 Biological Processes

In [11]:
pre_res_c1_go = run_prerank_analysis(c1_rnk, ["GO_Biological_Process_2021"], "results/prerank_c1_go", 1)
df_c1_go = process_results(pre_res_c1_go, "results/prerank_c1_go")

C2

In [12]:
pre_res_c2 = run_prerank_analysis(c2_rnk, GENE_SETS, "results/prerank_c2")
df_c2 = process_results(pre_res_c2, "results/prerank_c2")

2022-10-31 19:43:49,186 No supported gene_sets: GO_Biological_Process_2021MSigDB_Hallmark_2020


C2 Biological Processes

In [13]:
pre_res_c2_go = run_prerank_analysis(c2_rnk, ["GO_Biological_Process_2021"], "results/prerank_c2_go", 1)
df_c2_go = process_results(pre_res_c2_go, "results/prerank_c2_go")

## Prepare table with all results

In [14]:
df_c1["cluster"] = "cluster_1"
df_c1_go["cluster"] = "cluster_1"
df_c2["cluster"] = "cluster_2"
df_c2_go["cluster"] = "cluster_2"

df_full = pd.concat([df_c1, df_c1_go, df_c2, df_c2_go], axis=0)
df_full["cluster"].value_counts()

cluster_1    5089
cluster_2    3491
Name: cluster, dtype: int64

In [18]:
(
    df_full[[
        "cluster", "entrezgene_id", "external_gene_name", "Number of datasets", "max(|logFC|)", "logFC",
        "Term", "ES", "NES", "NOM p-val", "FDR q-val", "FWER p-val", "Tag %", "Gene %", "Lead_genes",
    ]].to_csv("results/Supp_Table4.tsv", sep='\t', index=False)
)